# **VISUALIZAÇÃO INTERATIVA DE CURVAS DE PERMANÊNCIA DE PRECIPITAÇÃO**

Para utilização deste caderno Jupyter deve-se executar cada uma das células pelo menos uma única vez e para gerar a curva de permanência para outras estações é só executar a `seção Curva de Permanência` novamente alterando o código da estação.

# Configurações

Configuração do Drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [0]:
import pandas as pd
import os
import datetime
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import Paired as palette
from google.colab import files
import sys
import folium
import itertools
output_notebook()
print('OK')

# Funções utilitárias

Funções de compatibilização das séries

In [3]:
def read_files(caminho):
    arquivos = os.listdir(caminho)
    ldf = []
    lkey = []
    for arquivo in arquivos:
        print(arquivo)
        file = caminho+'/'+arquivo
        dt = pd.read_csv(file, na_values=' ', sep=',', header=[0], engine='python', decimal='.')
        dt = dt.set_index('CODIGO')
        ldf.append(dt)
        key = arquivo.replace('.csv','')
        lkey.append(key)
    return dict(zip(lkey, ldf))

def get_station(code, listdf):
    series = {}
    for key in listdf:
        series[key] = listdf[key].loc[code]
    return series

def compat_time(dic):
    lmin = []
    lmax = []
    for key in dic:
        aux = dic[key].index.tolist()
        lmin.append(datetime.datetime.strptime(aux[0],'%d/%m/%Y'))
        lmax.append(datetime.datetime.strptime(aux[-1],'%d/%m/%Y'))
    lmin = sorted(lmin)
    lmax = sorted(lmax)
    vmin = lmin[-1]
    vmax = lmax[0]
    dtmin = str(vmin.day) +'/'+ str(vmin.month) +'/'+ str(vmin.year)
    dtmax = str(vmax.day) +'/'+ str(vmax.month) +'/'+ str(vmax.year)
    for key in dic:
        serie = dic[key]
        dic[key] = serie[dtmin:dtmax]
    return dic

def curva_permanencia(df,prod):
    ndf = df[prod]
    ndf = ndf.sort_values(ascending=False)
    ndf = ndf.reset_index()
    ndf = ndf.drop(columns=['index'])
    t = ndf.size + 1
    lista = list(range(1,t))
    ndf['indice'] = lista
    lper = []
    for i,row in ndf.iterrows():
        var = (ndf['indice'].iloc[i]/t)*100   
        lper.append(var)
    ndf['perm'] = lper
    return ndf
print('OK')

OK


Carregamento dos arquivos das séries de dados

In [0]:
data_dir = os.path.join('/content/gdrive', 'My Drive','Curva_Permanencia','dados')
dicdf = read_files(data_dir)
print('OK')

# Curva de Permanência

Escolha da estação e compatibilização das séries

In [0]:
estacao = int(input("Digite o código da estação pressione enter:")) #Exemplos de estacao = 936076 147015 2053000
print('OK')

In [0]:
series = get_station(estacao,dicdf) 
dicseries = compat_time(series)
df = pd.DataFrame(data=dicseries)
df = df.dropna()
col = df.columns
print('OK')

Cálculo da curva de permanência e plotagem do gráfico

In [0]:
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,crosshair"
p = figure(plot_width=600, plot_height=600,title="Curva de Permanência Estação: "+str(estacao),tools=TOOLS)
p.xaxis.axis_label = 'Permanência no Tempo (%)'
p.yaxis.axis_label = 'Precipitação Mensal (mm)'
colors = ['green','orange','blue','black','red','violet','peru','olive','coral','darkred','darkturquoise']
for prod,color in zip(col,colors):
    ndf = curva_permanencia(df,prod)
    p.line(ndf['perm'], ndf[prod], line_width=2,line_color=color,legend_label=prod.upper())
show(p)

Download da série da estação

In [0]:
name_file = 'serie_'+str(estacao)+'.csv'
df.to_csv(name_file)
files.download(name_file)
#describe
name_file2 = 'resumo_'+str(estacao)+'.csv'
desc = df.describe()
desc.to_csv(name_file2)
files.download(name_file2)